In [1]:
import pandas

In [3]:
data = pandas.read_csv("./california_schema.csv")
data.head()

,schema_name,table_name,column_name,data_type
0,california,buildings_a,gid,integer
1,california,buildings_a,osm_id,character varying(12)
2,california,buildings_a,code,smallint
3,california,buildings_a,fclass,character varying(28)
4,california,buildings_a,name,character varying(100)


In [9]:
data["table_name"].drop_duplicates().tolist()

['buildings_a',
 'landuse_a',
 'natural',
 'natural_a',
 'places',
 'places_a',
 'pofw',
 'pofw_a',
 'pois',
 'pois_a',
 'railways',
 'roads',
 'traffic',
 'traffic_a',
 'transport',
 'transport_a',
 'water_a',
 'waterways']

In [12]:
df = data.drop(columns=["schema_name"])
df.to_csv("california_schema_reduced.csv", index=False)
df.head()

,table_name,column_name,data_type
0,buildings_a,gid,integer
1,buildings_a,osm_id,character varying(12)
2,buildings_a,code,smallint
3,buildings_a,fclass,character varying(28)
4,buildings_a,name,character varying(100)


In [13]:
import pandas as pd
import os

# 读取 schema CSV 文件
schema_df = pd.read_csv("california_schema_reduced.csv")

# 创建存储 CSV 的目录
output_dir = "exported_tables"
os.makedirs(output_dir, exist_ok=True)

# 获取所有唯一的表名
unique_tables = schema_df["table_name"].unique()

# 记录表名和对应的 CSV 文件名
table_metadata = []

# 遍历每个表名并存储到独立的 CSV 文件中
for table in unique_tables:
    table_data = schema_df[schema_df["table_name"] == table]
    csv_filename = f"{output_dir}/{table}.csv"
    
    # 保存数据到 CSV
    table_data.to_csv(csv_filename, index=False)
    
    # 记录表名和 CSV 文件的映射
    table_metadata.append({"table_name": table, "csv_filename": csv_filename})

# 生成主 CSV，记录所有表的映射信息
metadata_df = pd.DataFrame(table_metadata)
metadata_df.to_csv(f"{output_dir}/table_metadata.csv", index=False)

print(f"✅ 所有表格数据已导出到 {output_dir} 目录，并生成了 table_metadata.csv")

✅ 所有表格数据已导出到 exported_tables 目录，并生成了 table_metadata.csv


In [14]:
import psycopg2

# 数据库连接信息
db_params = {
    "dbname": "chatgis",        # 数据库名
    "user": "chatgis",          # 用户名
    "password": "chatgis123", # 密码
    "host": "localhost",        # 数据库主机（如果在 Docker 里，可能是容器 IP）
    "port": "5432"              # 端口号
}

# 连接数据库
try:
    conn = psycopg2.connect(**db_params)
    cursor = conn.cursor()
    print("✅ Connected to PostGIS successfully!")

    # 执行查询示例
    cursor.execute("SELECT postgis_full_version();")
    print(cursor.fetchone())  # 输出 PostGIS 版本

    # 关闭连接
    cursor.close()
    conn.close()
except Exception as e:
    print(f"❌ Error: {e}")

✅ Connected to PostGIS successfully!
('POSTGIS="3.5.2 dea6d0a" [EXTENSION] PGSQL="170" GEOS="3.9.0-CAPI-1.16.2" PROJ="7.2.1 NETWORK_ENABLED=OFF URL_ENDPOINT=https://cdn.proj.org USER_WRITABLE_DIRECTORY=/var/lib/postgresql/.local/share/proj DATABASE_PATH=/usr/share/proj/proj.db" (compiled against PROJ 7.2.1) LIBXML="2.9.10" LIBJSON="0.15" LIBPROTOBUF="1.3.3" WAGYU="0.5.0 (Internal)" TOPOLOGY',)


In [16]:
import psycopg2
import csv

# 数据库连接配置
DB_CONFIG = {
    "dbname": "chatgis",
    "user": "chatgis",
    "password": "chatgis123",
    "host": "localhost",
    "port": "5432"
}

# 需要提取 fclass 的表
tables = ["buildings_a", "landuse_a", "natural", "natural_a", "places", "places_a", "pofw", "pofw_a", "pois", "pois_a"]

# 目标文件
output_file = "fclass_data.csv"

try:
    # 连接数据库
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    with open(output_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        
        # 遍历每个表
        for table in tables:
            query = f"SELECT DISTINCT fclass FROM california.{table} WHERE fclass IS NOT NULL;"
            cur.execute(query)
            rows = cur.fetchall()

            # 如果表有数据，写入表名及数据
            if rows:
                writer.writerow([table])  # 写入表名
                for row in rows:
                    writer.writerow(row)  # 写入 fclass 值

            print(table)

    print(f"数据已成功写入 {output_file}")

except Exception as e:
    print("发生错误:", e)

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()

buildings_a
landuse_a
natural
natural_a
places
places_a
pofw
pofw_a
pois
pois_a
数据已成功写入 fclass_data.csv


In [17]:
df = pd.read_csv("fclass_data.csv", encoding="utf-8")  # 也可以尝试 "latin1"
print(df.head())


  buildings_a
0    building
1   landuse_a
2  allotments
3    cemetery
4  commercial


In [21]:
import psycopg2
import pandas as pd

# 数据库连接配置
DB_CONFIG = {
    "dbname": "chatgis",
    "user": "chatgis",
    "password": "chatgis123",
    "host": "localhost",
    "port": "5432"
}

# 需要提取 fclass 的表
tables = [
    "buildings_a",
    "landuse_a",
    "natural",
    "natural_a",
    "places",
    "places_a",
    "pofw",
    "pofw_a",
    "pois",
    "pois_a",
    "railways",
    "roads",
    "traffic",
    "traffic_a",
    "transport",
    "transport_a",
    "water_a",
    "waterways"
]

# 存储数据的字典
fclass_counts = {}

try:
    # 连接数据库
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()

    # 遍历每个表，统计 fclass 频次
    for table in tables:
        query = f"SELECT fclass, COUNT(*) FROM california.{table} WHERE fclass IS NOT NULL GROUP BY fclass;"
        cur.execute(query)
        rows = cur.fetchall()

        # 记录数据
        fclass_counts[table] = {row[0]  for row in rows}

        print(f"Processed: {table}")

    # 转换为 DataFrame
    df = pd.DataFrame.from_dict(fclass_counts, orient="index").fillna(0)
    
    # 将 NaN 替换为 0，并转换数据类型
    df = df.fillna(0).astype(int)

    # 保存 CSV
    df.to_csv("fclass_counts.csv")

    print("数据已成功写入 fclass_counts.csv")

except Exception as e:
    print("发生错误:", e)

finally:
    if cur:
        cur.close()
    if conn:
        conn.close()

Processed: buildings_a
Processed: landuse_a
Processed: natural
Processed: natural_a
Processed: places
Processed: places_a
Processed: pofw
Processed: pofw_a
Processed: pois
Processed: pois_a
Processed: railways
Processed: roads
Processed: traffic
Processed: traffic_a
Processed: transport
Processed: transport_a
Processed: water_a
Processed: waterways
发生错误: invalid literal for int() with base 10: 'building'


In [22]:
df = pd.read_csv("fclass_counts.csv", encoding="utf-8")  # 也可以尝试 "latin1"
print(df.head())


    Unnamed: 0  building  allotments  cemetery  commercial  farmland  \
0  buildings_a   8915749           0         0           0         0   
1    landuse_a         0         233      1255       23309     22634   
2         pois         0           0         0           0         0   
3       pois_a         0           0         0           0         0   
4      natural         0           0         0           0         0   

   farmyard  forest  grass  heath  ...  waste_basket  wastewater_plant  \
0         0       0      0      0  ...             0                 0   
1      8082   35304  62259   3836  ...             0                 0   
2         0       0      0      0  ...          9111                 8   
3         0       0      0      0  ...           243              3097   
4         0       0      0      0  ...             0                 0   

   water_mill  water_tower  water_well  water_works  wayside_cross  \
0           0            0           0            0 